In [2]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools
from IPython.display import display
import os
import csv
from textblob import TextBlob

In [3]:
#Search term to be used on twitter scrape
search = "@subway"

#how to scrape
scraped_tweets= sntwitter.TwitterSearchScraper(search).get_items()
sliced_scraped_tweets = itertools.islice(scraped_tweets,10)

#create df of all the new data
df = pd.DataFrame(sliced_scraped_tweets)[['date', 'content']]

In [4]:
df

,date,content
0,2021-07-15 17:59:50+00:00,@SUBWAY \n\nFake tuna and shitty spokes people...
1,2021-07-15 17:59:38+00:00,If .@SUBWAY settles this case and claims they'...
2,2021-07-15 17:59:25+00:00,Why y’all ain’t tell me @SUBWAY don’t sell pea...
3,2021-07-15 17:56:52+00:00,"@SUBWAY 1st Jared, now it's no tuna in the tun..."
4,2021-07-15 17:55:48+00:00,@SUBWAY's new menu is the worst thing to ever ...
5,2021-07-15 17:54:33+00:00,"@4lisaguerrero @SUBWAY I saw it, i think they ..."
6,2021-07-15 17:54:07+00:00,"Tom Brady, Serena Williams, Steph Curry and ma..."
7,2021-07-15 17:52:41+00:00,@SUBWAY The new bread sucks! Hard and chewy!
8,2021-07-15 17:52:38+00:00,This going to be another day where @SUBWAY @Su...
9,2021-07-15 17:48:53+00:00,Leave my fucking @SUBWAY Tuna alone. I LOVE it...


In [5]:
#create array that will store the names of all the cities
city_array = []

#import dataframe containing populations of cities in california
california_cities_df = pd.read_csv('cal_populations_city.csv')


In [6]:
california_cities_df['pop_april_2010'].count()

455

In [7]:
california_cities_df.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
0,San Bernardino,Adelanto,1970,2164.0,8517,18130,31765
1,Los Angeles,Agoura Hills,1982,NaN,20390,20537,20330
2,Alameda,Alameda,1854,63852.0,76459,72259,73812
3,Alameda,Albany,1908,15130.0,16327,16444,18539
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089


In [8]:
#only want to look at cities bigger than 75k to ensure there is enough data
bigger_ca_cities_df = california_cities_df[california_cities_df['pop_april_2010']>75000] 

#rslt_df = dataframe[dataframe['Percentage'] > 70] 

In [9]:
bigger_ca_cities_df.count()

#100 cities meet the criteria

County                 100
City                   100
Incorportation_date    100
pop_april_1980          95
pop_april_1990         100
pop_april_2000         100
pop_april_2010         100
dtype: int64

In [10]:
#df of bigger cities in CA
bigger_ca_cities_df.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089
7,Orange,Anaheim,1876,219494.0,266406,328014,336265
10,Contra Costa,Antioch,1872,42683.0,62195,90532,102372
24,Kern,Bakersfield,1873,105611.0,174820,246889,347483
25,Los Angeles,Baldwin Park,1956,50554.0,69330,75837,75390


In [11]:
city_array = bigger_ca_cities_df['City']

In [12]:
#import dataframe containing city coordinates
city_latlng = pd.read_csv('cal_cities_lat_long.csv')

In [13]:
#rename columns
city_latlng = city_latlng.rename(columns={'Name': 'City', 'Latitude': 'lat', 'Longitude': 'lng'})

city_latlng.head()

,City,lat,lng
0,Adelanto,34.582769,-117.409214
1,Agoura Hills,34.153339,-118.761675
2,Alameda,37.765206,-122.241636
3,Albany,37.886869,-122.297747
4,Alhambra,34.095286,-118.127014


In [14]:
#merge columns together, so have city populations, and long/lat
city_df_merged = bigger_ca_cities_df.merge(city_latlng, how='inner', on='City')

In [15]:
#
city_df_merged.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010,lat,lng
0,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089,34.095286,-118.127014
1,Orange,Anaheim,1876,219494.0,266406,328014,336265,33.835292,-117.914503
2,Contra Costa,Antioch,1872,42683.0,62195,90532,102372,38.004922,-121.805789
3,Kern,Bakersfield,1873,105611.0,174820,246889,347483,35.373292,-119.018711
4,Los Angeles,Baldwin Park,1956,50554.0,69330,75837,75390,34.085286,-117.960897


In [16]:
#create df of just columns that are needed
final_city_df = city_df_merged[['City', 'pop_april_2010', 'lat', 'lng']]

In [17]:
final_city_df.count()

City              100
pop_april_2010    100
lat               100
lng               100
dtype: int64

In [18]:
final_city_df = final_city_df.rename(columns={'City':'city'})

In [19]:
final_city_df

,city,pop_april_2010,lat,lng
0,Alhambra,83089,34.095286,-118.127014
1,Anaheim,336265,33.835292,-117.914503
2,Antioch,102372,38.004922,-121.805789
3,Bakersfield,347483,35.373292,-119.018711
4,Baldwin Park,75390,34.085286,-117.960897
...,...,...,...,...
95,Visalia,124442,36.330228,-119.292058
96,Vista,93834,33.200036,-117.242536
97,West Covina,106098,34.068622,-117.938953
98,Westminster,89701,33.751342,-117.993992


In [20]:
final_city_len = final_city_df['lat'].count()

In [22]:
##tests
#lat = "37.765206"
#lng = "-122.241636"


df_array = []
i = 0
#cycle through every 
#while i < final_city_df['lat'].count():
    
#testing
while i < final_city_len:
    #creates a string for twitter scrapper to read and use as location
    loc = f"{str(final_city_df.loc[i, 'lat'])}, {str(final_city_df.loc[i, 'lng'])}, 10km"
    
    df_test = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    '@starbucks geocode:"{}"'.format(loc)).get_items(), 5000))[['date', 'content', 'user']]
    
    df_test['user_location'] = df_test['user'].apply(lambda x: x['location'])
    df_test['lat'] = final_city_df.loc[i, 'lat']
    df_test['lng'] = final_city_df.loc[i, 'lng']
    
    df_array.append(df_test)
    i = i +1

In [23]:
for i in df_array:
    display(i)

,date,content,user,user_location,lat,lng
0,2021-07-14 01:34:34+00:00,@Channel8ABC @BurgerKing @kfc @McDonalds @Wend...,"{'username': 'TravelBud76', 'id': 57561443, 'd...","Pasadena, CA",34.095286,-118.127014
1,2021-07-13 23:44:35+00:00,@walkingoogle @Starbucks redeem these 4500 sta...,"{'username': 'are_why_a_in', 'id': 390560316, ...","Los Angeles, CA",34.095286,-118.127014
2,2021-07-13 23:44:16+00:00,@_YeOldeCabin @Starbucks what about 4500 stars...,"{'username': 'are_why_a_in', 'id': 390560316, ...","Los Angeles, CA",34.095286,-118.127014
3,2021-07-13 23:43:56+00:00,@marinergirl75 @Starbucks but do you have 4500...,"{'username': 'are_why_a_in', 'id': 390560316, ...","Los Angeles, CA",34.095286,-118.127014
4,2021-07-13 23:43:26+00:00,@Starbucks will you share this on your main pa...,"{'username': 'are_why_a_in', 'id': 390560316, ...","Los Angeles, CA",34.095286,-118.127014
...,...,...,...,...,...,...
130,2020-12-24 19:25:26+00:00,"Working, but at least I got my iced coffee. 🏥😚...","{'username': 'Sinnthhia', 'id': 58986993, 'dis...","Montebello, CA",34.095286,-118.127014
131,2020-12-15 19:46:24+00:00,I think I deserve some @Starbucks. All my lab ...,"{'username': 'Sinnthhia', 'id': 58986993, 'dis...","Montebello, CA",34.095286,-118.127014
132,2020-12-09 16:11:09+00:00,"@starbucks #aintgotnothingonme @ Rosemead, Cal...","{'username': 'andrewvasquez83', 'id': 39179529...","Montebello, CA",34.095286,-118.127014
133,2020-12-04 22:34:29+00:00,Can you guys why I love this painting \n@starb...,"{'username': 'sunnylarue17', 'id': 10094815785...","Los Angeles, CA",34.095286,-118.127014


,date,content,user,user_location,lat,lng
0,2021-07-11 23:53:36+00:00,@bookiesncookies @Starbucks yeah that’s a good...,"{'username': 'alliewithbooks', 'id': 113856211...",Disneyland,33.835292,-117.914503
1,2021-07-11 21:40:09+00:00,so is @Starbucks charging for milk alternative...,"{'username': 'alliewithbooks', 'id': 113856211...",Disneyland,33.835292,-117.914503
2,2021-07-07 20:30:57+00:00,@Starbucks OMFG why is the CEO RUINING STARBUC...,"{'username': 'Excaligirl', 'id': 8644732, 'dis...","Norco, CA",33.835292,-117.914503
3,2021-07-07 15:09:24+00:00,@Starbucks True 🧊☕️ https://t.co/RBVQl6QJV9,"{'username': 'illuminEddiie', 'id': 274103858,...","Anaheim, CA",33.835292,-117.914503
4,2021-07-02 21:16:34+00:00,Did @Starbucks give up on green tea? It's gett...,"{'username': 'BigMac07', 'id': 32095920, 'disp...",The OC,33.835292,-117.914503
...,...,...,...,...,...,...
2517,2010-10-20 17:31:38+00:00,@starbucks! I LOVE my #neighborhoodstarbucks!!...,"{'username': 'CassieRussek', 'id': 31628020, '...",los angeles,33.835292,-117.914503
2518,2010-09-29 15:58:25+00:00,4 pump double shot @Starbucks chai latte to st...,"{'username': 'McVivicakes', 'id': 52949807, 'd...","Anaheim, CA",33.835292,-117.914503
2519,2010-09-25 16:24:59+00:00,"Car droped off car, walked to @starbucks, now ...","{'username': 'colorcopies', 'id': 15883869, 'd...","Santa ana, ca",33.835292,-117.914503
2520,2010-08-23 23:14:01+00:00,Driven out of @Starbucks by the smell of two h...,"{'username': 'TravisGadsden', 'id': 9272202, '...","Southern California, USA",33.835292,-117.914503


,date,content,user,user_location,lat,lng
0,2021-05-12 06:47:50+00:00,I’m looking foward to tomorrow mornings coffee...,"{'username': '_daniela_sarahi', 'id': 35545309...","Pittsburg, CA",38.004922,-121.805789
1,2021-03-03 03:12:25+00:00,@Starbucks just fucked up my drink,"{'username': 'kiaaarra_', 'id': 1674284749, 'd...","Vallejo, CA",38.004922,-121.805789
2,2021-01-24 23:42:18+00:00,"Yeah, @Starbucks is going meatless Mondays, wh...","{'username': 'vsansome', 'id': 24806248, 'disp...","California, USA",38.004922,-121.805789
3,2020-12-05 22:29:13+00:00,@daniellemquinn @Starbucks @asuonline Great jo...,"{'username': 'cojfootball', 'id': 43348630, 'd...",Brentwood (NorCal),38.004922,-121.805789
4,2020-09-08 23:25:25+00:00,Tastes like Christmas morning ... well done @S...,"{'username': 'JohnNetworQ', 'id': 953515440, '...",Awawo,38.004922,-121.805789
...,...,...,...,...,...,...
175,2012-01-31 14:15:45+00:00,@Starbucks Last Day of January! Thanks for the...,"{'username': 'lecramdrawde', 'id': 19827696, '...","Manteca, CA",38.004922,-121.805789
176,2012-01-14 21:54:28+00:00,Got coffee on my mind. :0) @ChooseCoffee @hash...,"{'username': 'SWEETLiSAxo', 'id': 458600022, '...","BAY AREA, CALIFORNIA",38.004922,-121.805789
177,2011-12-07 16:06:41+00:00,@Starbucks I love the holiday drinks you guys ...,"{'username': 'OchibiSamaDesu', 'id': 91232101,...","Antioch, CA",38.004922,-121.805789
178,2011-09-10 17:44:55+00:00,@Starbucks I'm glad u were nearby while waitin...,"{'username': 'Bevstwit', 'id': 178518054, 'dis...",SF Bay Area,38.004922,-121.805789


,date,content,user,user_location,lat,lng
0,2021-07-12 22:25:00+00:00,@Ladysc23 @Starbucks Right! I’ve been annoyed ...,"{'username': 'Nezzie2203', 'id': 3418725554, '...",,35.373292,-119.018711
1,2021-07-10 01:49:41+00:00,@Starbucks I’ve ordered 3 vsc cold brews since...,"{'username': 'JuIieaux', 'id': 2760934646, 'di...","Bakersfield, CA",35.373292,-119.018711
2,2021-06-21 21:30:40+00:00,What happens when you write by the seat of you...,"{'username': 'wetzelv', 'id': 1247789905, 'dis...","West Sacramento, CA",35.373292,-119.018711
3,2021-04-30 22:51:53+00:00,@Starbucks Clydie loves Puppaccinos ❣️🐕 #ThePu...,"{'username': 'roze_hurd', 'id': 2747035544, 'd...","Las Vegas, NV",35.373292,-119.018711
4,2021-04-09 00:44:49+00:00,@jonathancivil @Starbucks I’m so sorry 😂,"{'username': 'MrJoseRozay', 'id': 911656604570...",CA/TX,35.373292,-119.018711
...,...,...,...,...,...,...
338,2011-01-11 20:15:47+00:00,"to @Starbucks I had oatmeal with nuts, raisins...","{'username': 'Kevinmartinelli', 'id': 14329633...","Bakersfield, California",35.373292,-119.018711
339,2010-12-25 19:22:54+00:00,@Starbucks thanks to all your hard workers on ...,"{'username': 'karb0n13', 'id': 15382687, 'disp...",California,35.373292,-119.018711
340,2010-10-21 20:55:05+00:00,Gonna take my Mala Onda to @Starbucks buy a Ba...,"{'username': 'Milchito', 'id': 32808372, 'disp...","Miami Beach, FL",35.373292,-119.018711
341,2010-10-19 23:16:28+00:00,just paid for his coffee with his iPhone! @Sta...,"{'username': 'pittsspotter', 'id': 19304330, '...","Nairobi, Kenya",35.373292,-119.018711


,date,content,user,user_location,lat,lng
0,2021-07-12 20:31:42+00:00,@Starbucks what can you tell us about 🦉?,"{'username': 'Shahabs', 'id': 33845458, 'displ...","Pasadena, CA",34.085286,-117.960897
1,2021-07-12 20:28:29+00:00,@Starbucks @awils165 Has to be #safemoon related,"{'username': 'Shahabs', 'id': 33845458, 'displ...","Pasadena, CA",34.085286,-117.960897
2,2021-07-08 22:10:36+00:00,@EAWilsonAuthor Or maybe a free @Starbucks co...,"{'username': 'DSiflingWCPDPIO', 'id': 71734127...","West Covina, CA",34.085286,-117.960897
3,2021-07-03 20:35:26+00:00,@ohheytommy @Starbucks I want cinnamon dolce l...,"{'username': 'Mrs_Ayano', 'id': 13038883881440...",,34.085286,-117.960897
4,2021-06-30 19:42:54+00:00,@Starbucks do you have any coupons foe next week?,"{'username': 'NotNewTwiterite', 'id': 12277639...",,34.085286,-117.960897
...,...,...,...,...,...,...
1152,2010-12-02 17:04:26+00:00,"I'm @starbucks (5705 Rosemead Blvd, Temple Cit...","{'username': 'cat1152114', 'id': 30341217, 'di...","ÜT: 34.129889,-118.069719",34.085286,-117.960897
1153,2010-11-14 22:49:48+00:00,"I'm @starbucks (733 W Naomi Ave, Arcadia) htt...","{'username': 'cat1152114', 'id': 30341217, 'di...","ÜT: 34.129889,-118.069719",34.085286,-117.960897
1154,2010-11-01 14:23:31+00:00,Breakfast @starbucks gotta have my coffee  to...,"{'username': 'OpTicSwagg24', 'id': 191716219, ...",,34.085286,-117.960897
1155,2010-09-28 22:19:40+00:00,This looks very exciting.... @Starbucks http:/...,"{'username': 'JeffDMartinez', 'id': 21318642, ...","Nashville, TN",34.085286,-117.960897


...

,date,content,user,user_location,lat,lng
0,2021-06-24 15:01:49+00:00,Quad espresso to help with this day @Starbucks...,"{'username': 'DTPMZP', 'id': 97618660, 'displa...",United States,36.330228,-119.292058
1,2020-12-11 17:09:21+00:00,@Starbucks https://t.co/tZ78j6DTEs,"{'username': 'alleenagonzalez', 'id': 25727856...","California, USA",36.330228,-119.292058
2,2020-09-24 17:02:00+00:00,@misomaichi @Starbucks Omg what did you order ...,"{'username': 'youpinsee', 'id': 4553864773, 'd...","California, USA",36.330228,-119.292058
3,2020-09-21 23:22:56+00:00,I Went to 2 different @Starbucks locations tod...,"{'username': 'Ryyahh23', 'id': 979305966, 'dis...","California, USA",36.330228,-119.292058
4,2020-07-03 02:18:52+00:00,@Starbucks @thekpopaccount Hyungwon has Starbu...,"{'username': 'monizzy0618', 'id': 166336580, '...",,36.330228,-119.292058
...,...,...,...,...,...,...
99,2012-01-20 18:58:53+00:00,There's nothing like a shot of espresso and an...,"{'username': 'gavenoakley', 'id': 15436449, 'd...",United States,36.330228,-119.292058
100,2011-11-16 05:19:04+00:00,Peppermint White Chocolate Mocha at @Starbucks...,"{'username': 'gavenoakley', 'id': 15436449, 'd...",United States,36.330228,-119.292058
101,2011-11-12 07:11:35+00:00,@Starbucks Just found this at my local @Target...,"{'username': 'gavenoakley', 'id': 15436449, 'd...",United States,36.330228,-119.292058
102,2011-11-12 01:43:54+00:00,"Yeah, I love @Starbucks enough to walk half a ...","{'username': 'TaylorCharbjklz', 'id': 25527136...",Richmond,36.330228,-119.292058


,date,content,user,user_location,lat,lng
0,2021-07-13 23:37:08+00:00,So yesterday I tried @Starbucks grilled cheese...,"{'username': 'Jackie135_7', 'id': 591043386, '...","San Marcos,CA",33.200036,-117.242536
1,2021-07-11 02:44:59+00:00,@CherishToday08 @Starbucks Bond 💜🥂 @MissSally...,"{'username': 'LaneBaima', 'id': 88441590665424...","Carlsbad, CA",33.200036,-117.242536
2,2021-07-08 05:30:28+00:00,@Starbucks what gives? wanted to pay $DOGE for...,"{'username': 'daviddelgado', 'id': 17050943, '...","Carlsbad, CA",33.200036,-117.242536
3,2021-07-07 00:10:08+00:00,@jmjohn15 @CherishToday08 @FetchTeach @Gabriel...,"{'username': 'LaneBaima', 'id': 88441590665424...","Carlsbad, CA",33.200036,-117.242536
4,2021-07-06 15:19:33+00:00,@Starbucks Good idea 👍🏼,"{'username': 'LaneBaima', 'id': 88441590665424...","Carlsbad, CA",33.200036,-117.242536
...,...,...,...,...,...,...
533,2011-03-09 22:02:11+00:00,Teaching @jamesdoezie how to skateboard on the...,"{'username': 'corey_dylan', 'id': 35427941, 'd...",Ocean Beach | San Diego,33.200036,-117.242536
534,2010-11-09 16:16:57+00:00,"http://twitpic.com/35cmt3 OceansideDMV, only p...","{'username': 'NickyChips', 'id': 25372542, 'di...",OutsideTheBox : InsideTheLines,33.200036,-117.242536
535,2010-07-25 16:52:50+00:00,Kicking it at Carlsbad @Starbucks with Hawaii ...,"{'username': 'NickyChips', 'id': 25372542, 'di...",OutsideTheBox : InsideTheLines,33.200036,-117.242536
536,2010-06-03 17:45:50+00:00,@Starbucks I used my rewards card 5+ times bef...,"{'username': 'Neufusion', 'id': 16655831, 'dis...","Vista, CA",33.200036,-117.242536


,date,content,user,user_location,lat,lng
0,2021-07-12 20:31:42+00:00,@Starbucks what can you tell us about 🦉?,"{'username': 'Shahabs', 'id': 33845458, 'displ...","Pasadena, CA",34.068622,-117.938953
1,2021-07-12 20:28:29+00:00,@Starbucks @awils165 Has to be #safemoon related,"{'username': 'Shahabs', 'id': 33845458, 'displ...","Pasadena, CA",34.068622,-117.938953
2,2021-07-08 22:10:36+00:00,@EAWilsonAuthor Or maybe a free @Starbucks co...,"{'username': 'DSiflingWCPDPIO', 'id': 71734127...","West Covina, CA",34.068622,-117.938953
3,2021-07-03 20:35:26+00:00,@ohheytommy @Starbucks I want cinnamon dolce l...,"{'username': 'Mrs_Ayano', 'id': 13038883881440...",,34.068622,-117.938953
4,2021-06-30 19:42:54+00:00,@Starbucks do you have any coupons foe next week?,"{'username': 'NotNewTwiterite', 'id': 12277639...",,34.068622,-117.938953
...,...,...,...,...,...,...
1080,2010-12-02 03:03:20+00:00,@gregmaron & I are leaving @starbucks because ...,"{'username': 'AdamGoyer', 'id': 104438475, 'di...","Los Angeles, CA",34.068622,-117.938953
1081,2010-11-19 19:05:36+00:00,"Man, @Starbucks has a strong business client b...","{'username': 'AdamGoyer', 'id': 104438475, 'di...","Los Angeles, CA",34.068622,-117.938953
1082,2010-11-01 14:23:31+00:00,Breakfast @starbucks gotta have my coffee  to...,"{'username': 'OpTicSwagg24', 'id': 191716219, ...",,34.068622,-117.938953
1083,2010-09-28 22:19:40+00:00,This looks very exciting.... @Starbucks http:/...,"{'username': 'JeffDMartinez', 'id': 21318642, ...","Nashville, TN",34.068622,-117.938953


,date,content,user,user_location,lat,lng
0,2021-07-11 23:53:36+00:00,@bookiesncookies @Starbucks yeah that’s a good...,"{'username': 'alliewithbooks', 'id': 113856211...",Disneyland,33.751342,-117.993992
1,2021-07-11 21:40:09+00:00,so is @Starbucks charging for milk alternative...,"{'username': 'alliewithbooks', 'id': 113856211...",Disneyland,33.751342,-117.993992
2,2021-07-11 16:52:53+00:00,@catheri88438842 @DakIsMyQB1 @GarDolphin733 @l...,"{'username': 'jonathan_shen7', 'id': 550933750...","Huntington Beach, California",33.751342,-117.993992
3,2021-07-11 16:01:16+00:00,A @Starbucks hack for Vietnamese Iced Coffee e...,"{'username': 'foodbeast', 'id': 17382260, 'dis...","Santa Ana, CA",33.751342,-117.993992
4,2021-07-11 15:06:04+00:00,@catheri88438842 @DakIsMyQB1 @GarDolphin733 @l...,"{'username': 'jonathan_shen7', 'id': 550933750...","Huntington Beach, California",33.751342,-117.993992
...,...,...,...,...,...,...
1599,2010-09-29 15:58:25+00:00,4 pump double shot @Starbucks chai latte to st...,"{'username': 'McVivicakes', 'id': 52949807, 'd...","Anaheim, CA",33.751342,-117.993992
1600,2010-09-25 16:24:59+00:00,"Car droped off car, walked to @starbucks, now ...","{'username': 'colorcopies', 'id': 15883869, 'd...","Santa ana, ca",33.751342,-117.993992
1601,2010-08-08 19:00:16+00:00,Is grabbing some @Starbucks,"{'username': 'fabulousjoshie', 'id': 74038032,...",From Small town Now Cincinnati,33.751342,-117.993992
1602,2010-07-02 00:28:51+00:00,@Starbucks =]],"{'username': 'johnymilaphoto', 'id': 111200627...","Los Angeles, CA",33.751342,-117.993992


,date,content,user,user_location,lat,lng
0,2021-07-12 03:34:28+00:00,@loqua_renee @Starbucks Ohhhh I believe that. ...,"{'username': 'Joel_Simpson', 'id': 67280132, '...","Whittier, CA",33.979178,-118.032844
1,2021-07-12 02:33:32+00:00,@tobybrooke @Starbucks Well that too. 🤣,"{'username': 'Joel_Simpson', 'id': 67280132, '...","Whittier, CA",33.979178,-118.032844
2,2021-07-12 02:25:06+00:00,We all know that churches spend all their unta...,"{'username': 'Joel_Simpson', 'id': 67280132, '...","Whittier, CA",33.979178,-118.032844
3,2021-07-06 16:43:57+00:00,What’s up with @Starbucks using way to much cr...,"{'username': 'Mercabet', 'id': 528834338, 'dis...","Downey, CA",33.979178,-118.032844
4,2021-06-20 18:28:04+00:00,whoever came up with the medicine ball at @Sta...,"{'username': 'chrispychriscat', 'id': 39141596...","Pasadena, CA",33.979178,-118.032844
5,2021-06-16 20:19:38+00:00,@crocs goes to @starbucks …perfect combo 🤟🏼 🍍 ...,"{'username': 'ForeverAmber3', 'id': 28901170, ...",California,33.979178,-118.032844
6,2021-05-29 16:05:01+00:00,@imbradmiller @Starbucks Yup I have 3 of them ...,"{'username': 'jasontucker', 'id': 791125, 'dis...","Whittier, CA 90603",33.979178,-118.032844


In [24]:
print(loc)

33.979178, -118.032844, 10km


In [51]:
df_test = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    '@McDonalds geocode:"{}"'.format(loc)).get_items(), 1000))[['date', 'content']]

In [25]:
final_df_array = []

for df in df_array:
    print(df['date'].count())
    if df['date'].count() > 30:
        final_df_array.append(df)
        


135
2522
180
343
1157
70
161
1612
4
493
1476
107
231
69
645
43
403
288
1569
3108
44
518
16
301
61
582
23
565
2158
3020
122
60
395
75
106
1389
226
5000
1300
93
144
175
468
2908
92
646
306
41
125
1204
19
1068
359
840
1871
441
97
10
725
624
69
985
475
255
239
415
888
140
593
55
4132
1672
637
593
566
2585
233
1284
421
95
5000
149
3
59
254
885
268
22
1139
81
1897
72
36
126
100
104
538
1085
1604
7


In [110]:
for df in final_df_array:
    print(df['date'].count())

115
1192
46
218
842
372
239
966
46
49
394
48
83
504
1422
44
245
173
71
50
32
392
1153
1233
44
31
35
75
146
4606
344
54
64
100
138
74
46
162
37
363
91
214
423
506
204
38
237
370
358
136
76
93
114
306
69
339
67
1255
1730
370
231
681
119
1783
229
75
3756
70
186
125
151
658
109
485
31
68
59
302
778
416


In [26]:
rows = []

for df in final_df_array:
    sentiment_total = 0
    df_len = df['date'].count()
    i = 0
    while i < df_len:
        testimonial = TextBlob(df.loc[i, 'content'])
        sentiment_total = sentiment_total + testimonial.sentiment.polarity
        sentiment_average = sentiment_total/df_len
        i = i+1
    print(f"({df.loc[1,'lat']}, {df.loc[1,'lng']}): Sentiment Total = {sentiment_total}, Average Sentiment: {sentiment_average}")
    rows.append([df.loc[1,'lat'],df.loc[1,'lng'], sentiment_total, sentiment_average])
    print("-" *42)

(34.095286, -118.127014): Sentiment Total = 13.092027163374818, Average Sentiment: 0.09697797898796161
------------------------------------------
(33.835292, -117.914503): Sentiment Total = 360.40296882456875, Average Sentiment: 0.142903635537101
------------------------------------------
(38.004922, -121.805789): Sentiment Total = 22.203862433862433, Average Sentiment: 0.12335479129923574
------------------------------------------
(35.373292, -119.018711): Sentiment Total = 34.03192392676766, Average Sentiment: 0.0992184371042789
------------------------------------------
(34.085286, -117.960897): Sentiment Total = 125.3082766936478, Average Sentiment: 0.10830447423824356
------------------------------------------
(33.881683, -118.117011): Sentiment Total = 6.127256944444444, Average Sentiment: 0.08753224206349206
------------------------------------------
(37.871592, -122.272747): Sentiment Total = 6.10054879148629, Average Sentiment: 0.03789160740053596
-----------------------------

(37.935758, -122.34775): Sentiment Total = 21.963777958152946, Average Sentiment: 0.08613246258099194
------------------------------------------
(33.95335, -117.396156): Sentiment Total = 17.268213383838386, Average Sentiment: 0.07225193884451207
------------------------------------------
(38.752122, -121.288006): Sentiment Total = 50.26636582745957, Average Sentiment: 0.12112377307821584
------------------------------------------
(38.581572, -121.4944): Sentiment Total = 87.90333578024985, Average Sentiment: 0.09899024299577686
------------------------------------------
(36.677736, -121.6555): Sentiment Total = 15.619268071338382, Average Sentiment: 0.11156620050955987
------------------------------------------
(34.108344, -117.289764): Sentiment Total = 68.50174047182976, Average Sentiment: 0.11551726892382759
------------------------------------------
(32.715328, -117.157256): Sentiment Total = 10.860898476523477, Average Sentiment: 0.19747088139133595
------------------------------

In [27]:
total_sentiment_df = pd.DataFrame(rows, columns=["lat", "lng", "sentiment_total", "sentiment_average"])

In [28]:
total_sentiment_df

,lat,lng,sentiment_total,sentiment_average
0,34.095286,-118.127014,13.092027,0.096978
1,33.835292,-117.914503,360.402969,0.142904
2,38.004922,-121.805789,22.203862,0.123355
3,35.373292,-119.018711,34.031924,0.099218
4,34.085286,-117.960897,125.308277,0.108304
...,...,...,...,...
87,34.536108,-117.291158,9.053885,0.090539
88,36.330228,-119.292058,17.373688,0.167055
89,33.200036,-117.242536,67.138283,0.124792
90,34.068622,-117.938953,116.945719,0.107784


In [29]:
total_sentiment_df.to_csv("starbucks_tweets_average_sentiments.csv")


In [108]:
# Specify the file to write to
output_path = os.path.join("..","tweet_csvs", "mcdonalds_tweets.csv")

# Open the file using "write" mode. Specify the variable to hold the contents
with open(output_path, 'w', newline='') as csvfile:
    
    # Initialize csv.writer
    csvwriter = csv.writer(csvfile, delimiter=',')
    
    csvwriter.writerow(['date', 'content', 'user_location', 'lat', 'lng'])
    
    for df in final_df_array:
        df_len = df['date'].count()
        i = 0
        while i < df_len:            
            csvwriter.writerow([df.loc[i, 'date'], df.loc[i, 'content'], df.loc[i, 'user_location'], df.loc[i, 'lat'], df.loc[i,'lng']])
            i = i+1
    # Write the first row (column headers)

    #date	content	user	user_location	lat	lng